# HW3
This program retrieves eia government API data and writes it to a sqlite3 db.

In [1]:
import config
import sqlite3
import requests
import pandas as pd

In [2]:
# get data
EIA_API_KEY = config.EIA_API_KEY
API_ROUTE = config.EIA_API_ROUTE
data = None
try:
    response = requests.get(API_ROUTE, headers={"X-Api-Key": EIA_API_KEY})
    response.raise_for_status()  # Raise an HTTPError for bad responses
    data = response.json()
    data = data['response']['data']
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")

In [14]:
# put the data into a pandas dataframe
if data is None:
    print("Data is None")
    exit()
df = pd.DataFrame(data)
# Sort by time period
df_sorted = df.sort_values(by='period', ascending=False)
# print(df_sorted.head())

In [15]:
# write to sqlite
conn = sqlite3.connect('energy_data.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS energy_data (
                    country TEXT,
                    period TEXT,
                    energy_source TEXT,
                    value REAL,
                    unit TEXT)''')

for _, row in df.iterrows():
    value = row['value'] 
    
    if value == 'w' or value == '--':
        value = None
        
    cursor.execute('''INSERT INTO energy_data (country, period, energy_source, value, unit) 
                        VALUES (?, ?, ?, ?, ?)''', 
                        (row['countryRegionName'], row['period'], row['productName'], value, row['unit']))

conn.commit()
conn.close()

In [3]:
# query the data, getting the most popular energy source per year for the US
conn = sqlite3.connect('energy_data.db')
cursor = conn.cursor()

cursor.execute('''SELECT period, energy_source, MAX(value) as max_value
                    FROM energy_data
                    WHERE country = 'United States'
                    GROUP BY period''')

rows = cursor.fetchall()
for row in rows:
    print(row)
    
conn.close()

('2024-04', 'Total petroleum and other liquids', 22719.646)
('2024-05', 'Total petroleum and other liquids', 22737.069)
('2024-06', 'Total petroleum and other liquids', 22810.883)
